In [1]:
import pandas as pd
from pathlib import Path
import re

In [59]:
big_path = "../../../Traffic_Violations.csv"
path= "sample_data.csv"
df = pd.read_csv(path, low_memory=False)
big_df = pd.read_csv(big_path, low_memory=False)

In [27]:
df.head()

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,971ef50f-f138-419f-89e5-5d2cc5d7b75a,04/30/2023,23:55:00,MCP,"4th District, Wheaton",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,KEMP MILL ROAD @ ALPERT LANE,39.053038,-77.024637,No,...,13-401(h),Transportation Article,False,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0530383333333, -77.0246366666667)"
1,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",NEGLIGENT DRIVING VEHICLE IN CARELESS AND IMPR...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-901.1(b),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"
2,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",RECKLESS DRIVING VEHICLE IN WANTON AND WILLFUL...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-901.1(a),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"
3,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",FAILURE TO DRIVE VEHICLE ON RIGHT HALF OF ROAD...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-301(a),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"
4,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",DRIVING VEHICLE WHILE UNDER THE INFLUENCE OF A...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-902(a1i),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"


In [6]:
df.columns


Index(['SeqID', 'Date Of Stop', 'Time Of Stop', 'Agency', 'SubAgency',
       'Description', 'Location', 'Latitude', 'Longitude', 'Accident', 'Belts',
       'Personal Injury', 'Property Damage', 'Fatal', 'Commercial License',
       'HAZMAT', 'Commercial Vehicle', 'Alcohol', 'Work Zone',
       'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Reason For Stop', 'Search Type',
       'Search Arrest Reason', 'State', 'VehicleType', 'Year', 'Make', 'Model',
       'Color', 'Violation Type', 'Charge', 'Article',
       'Contributed To Accident', 'Race', 'Gender', 'Driver City',
       'Driver State', 'DL State', 'Arrest Type', 'Geolocation'],
      dtype='object')

In [60]:
big_df_smaller = big_df.drop(["Agency", "Geolocation", "VehicleType", "HAZMAT", 
                                "Commercial Vehicle", "Commercial License", "Article", "Charge", 
                                "SeqID", "SubAgency", "Model", "Contributed To Accident", "Driver State", 
                                "Driver City", "Arrest Type", "Search Reason For Stop"], axis = 1)

In [25]:
big_df_smaller.columns
# citations_warnings = big_data_smaller[big_data_smaller["Violation Type"] = ]



Index(['Date Of Stop', 'Time Of Stop', 'Description', 'Location', 'Accident',
       'Belts', 'Personal Injury', 'Property Damage', 'Fatal', 'Alcohol',
       'Work Zone', 'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Type', 'Search Arrest Reason', 'State', 'Year',
       'Make', 'Color', 'Violation Type', 'Race', 'Gender', 'DL State',
       'Geolocation'],
      dtype='object')

In [61]:
# Drop rows that do not result in a citation or warning
big_df_smaller.drop(big_df_smaller[big_df_smaller['Violation Type'] == 'ESERO'].index, inplace=True)
big_df_smaller.drop(big_df_smaller[big_df_smaller['Violation Type'] == 'SERO'].index, inplace=True)



In [62]:
# Clean the Year column
# Drop rows with null year
big_df_smaller = big_df_smaller.dropna(subset=["Year"])
# Change year to integer
big_df_smaller["Year"]= big_df_smaller["Year"].astype(int)
# Get years in a list
years = big_df_smaller["Year"].unique()
# Make a list of garbage years
years_to_remove = [0, 6338, 1005, 1196, 2912, 1009, 2088, 1007, 2102, 2109, 2105, 2997]
# Clean the list of years
good_years = [value for value in years if value not in years_to_remove]
# Keep only rows with good years
big_df_smaller = big_df_smaller[big_df_smaller["Year"].isin(good_years)]
big_df_smaller.shape

(1788513, 27)

In [63]:
big_df_smaller["Search Reason"].value_counts()

Incident to Arrest        47292
Probable Cause            20671
Consensual                11528
K-9                        1613
Other                      1029
Exigent Circumstances       500
Probable Cause for CDS        4
Arrest/Tow                    3
plain view marijuana          3
DUI                           1
Name: Search Reason, dtype: int64

In [ ]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*EXCEEDING.*SPEED LIMIT.*', flags=re.IGNORECASE)

df['Description'] = df['Description'].apply(lambda x: re.sub(pattern, 'Speeding', x))